In [2]:
# Version 1 This is just the base program
import pygame
import numpy as np
import sys
import random

# Parameters
WIDTH, HEIGHT = 1800, 1000
NUM_PEOPLE = 50
PERSON_RADIUS = 5
MAX_SPEED = 1.38
SEPARATION_RADIUS = 25
WALL_MARGIN = 10

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)  # For path waypoints
BLUE = (0, 0, 255)   # For passable walls

# Pygame setup
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()

class Person:
    # Variables iniciales de la clase Person
    def __init__(self, x, y):
        self.position = np.array([x, y], dtype=float)
        self.velocity = np.random.rand(2) * MAX_SPEED
        self.radius = PERSON_RADIUS
        self.path_index = 0
        self.path_threshold = 20  # Distance to consider waypoint reached
    
    def follow_path(self, path_waypoints):
        if self.path_index < len(path_waypoints):
            target = path_waypoints[self.path_index]
            direction = target - self.position
            distance = np.linalg.norm(direction)
            
            if distance < self.path_threshold:
                self.path_index = (self.path_index + 1) % len(path_waypoints)  # Loop path
            else:
                # Normalize direction and apply to velocity
                if distance > 0:
                    direction_normalized = direction / distance
                    self.velocity += direction_normalized * 0.1
    
    def update(self, people, walls, passable_walls):
        # Follow the path, Force # 1
        self.follow_path(path_waypoints)
        
        # Separation from other people, Force # 2
        separation = np.zeros(2)
        # Se itera entre todas las instancias menos la que se está analizando
        for other in people:
            if other != self:
                dist = np.linalg.norm(self.position - other.position) # Se calcula la distancia
                if dist < SEPARATION_RADIUS: # Si la distancia de separación es menor que el límite permitido, se aleja
                    separation += (self.position - other.position) / (dist + 0.0001)  # Avoid division by zero
        
        # Wall avoidance (only for solid walls)
        # Se itera entre todas las paredes
        for wall in walls:
            dist_to_wall = self.distance_to_wall(wall)
            # Si la distancia es menor a la permitida se separa de la pared.
            if dist_to_wall < WALL_MARGIN:
                separation += (self.position - wall[:2]) / (dist_to_wall + 0.0001)
        
        # Pass through passable walls (no avoidance)
        # We could add special behavior here if needed
        
        # Update velocity and position
        self.velocity += separation * 0.1
        speed = np.linalg.norm(self.velocity)
        if speed > MAX_SPEED:
            self.velocity = (self.velocity / speed) * MAX_SPEED
        self.position += self.velocity
        
        # Keep within screen bounds
        self.position = np.clip(self.position, [0, 0], [WIDTH, HEIGHT])
    
    def distance_to_wall(self, wall):
        x1, y1, x2, y2 = wall
        px, py = self.position
        line_length = np.linalg.norm(np.array([x2 - x1, y2 - y1]))
        if line_length == 0:
            return np.linalg.norm(self.position - np.array([x1, y1]))
        t = max(0, min(1, np.dot([px - x1, py - y1], [x2 - x1, y2 - y1]) / line_length**2))
        closest_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
        return np.linalg.norm(self.position - closest_point)
    
    def draw(self, screen):
        pygame.draw.circle(screen, RED, self.position.astype(int), self.radius)

# Define path waypoints (green)
path_waypoints = [# A vector is define like this [x1, y1]
    np.array([200, 100]),
    np.array([1600, 200]),
    np.array([1600, 800]),
    np.array([200, 800])
]

# Define walls (black - solid, must avoid)
walls = [# A vector is define like this [x1, y1, x2, y2]
    # Screen boundaries
    [0, 0, WIDTH, 0],        # Top wall
    [WIDTH, 0, WIDTH, HEIGHT],  # Right wall
    [WIDTH, HEIGHT, 0, HEIGHT],  # Bottom wall
    [0, HEIGHT, 0, 0],        # Left wall
    
    # Interior walls with passages
    [0, 400, 800, 400],      # Horizontal wall left
    [1000, 400, WIDTH - 150, 400], # Horizontal wall right (gap between 800-1000)
    
    #[900, 400, 900, 0],      # Vertical wall top
    #[900, 600, 900, HEIGHT]  # Vertical wall bottom (gap between 400-600)
]

# Define passable walls (blue - can walk through)
passable_walls = [
#    [800, 400, 1000, 400],   # Passage between horizontal walls
#    [900, 400, 900, 600]     # Passage between vertical walls
]

# Create people
people = [Person(random.randint(0, WIDTH), random.randint(0, HEIGHT)) 
          for _ in range(NUM_PEOPLE)]

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    screen.fill(WHITE)
    
    # Draw path waypoints
    for i, point in enumerate(path_waypoints):
        pygame.draw.circle(screen, GREEN, point.astype(int), 10)
        if i > 0:
            pygame.draw.line(screen, (200, 200, 200), 
                           path_waypoints[i-1], point, 2)
    
    # Draw walls (solid)
    for wall in walls:
        pygame.draw.line(screen, BLACK, (wall[0], wall[1]), (wall[2], wall[3]), 5)
    
    # Draw passable walls (can walk through)
    for wall in passable_walls:
        pygame.draw.line(screen, BLUE, (wall[0], wall[1]), (wall[2], wall[3]), 3)
    
    # Update and draw people
    for person in people:
        person.update(people, walls, passable_walls)
        person.draw(screen)
    
    pygame.display.flip()
    clock.tick(60)

pygame.quit()
sys.exit()

SystemExit: 

In [32]:
# Version 1.2 This is the program with only the interval timing
import pygame
import numpy as np
import sys
import random
from pygame.locals import *

# Parameters
WIDTH, HEIGHT = 1800, 1000
NUM_PEOPLE = 100
PERSON_RADIUS = 5
MAX_SPEED = 2
SEPARATION_RADIUS = 20
WALL_MARGIN = 10
SPAWN_AREA = (20, 150, 20, 100)  # x_min, x_max, y_min, y_max
GROUP_SIZE = 10                   # Number of people per group
SPAWN_INTERVAL = 2000            # Milliseconds between group spawns

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255, 128)

# Pygame setup
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
font = pygame.font.SysFont('Arial', 20)

class Person:
    def __init__(self, x, y):
        self.position = np.array([x, y], dtype=float)
        self.velocity = np.random.rand(2) * MAX_SPEED
        self.radius = PERSON_RADIUS
        self.path_index = 0
        self.path_threshold = 20
        self.active = True
    
    def is_in_passage(self, position):
        for wall in passable_walls:
            x1, y1, x2, y2 = wall
            if (min(x1,x2)) <= position[0] <= max(x1,x2) and (min(y1,y2)) <= position[1] <= max(y1,y2):
                return True
        return False
    
    def follow_path(self, path_waypoints):
        if self.path_index < len(path_waypoints):
            target = path_waypoints[self.path_index]
            direction = target - self.position
            distance = np.linalg.norm(direction)
            
            if distance < self.path_threshold:
                self.path_index = (self.path_index + 1) % len(path_waypoints)
            elif distance > 0:
                direction_normalized = direction / distance
                self.velocity += direction_normalized * 0.1
    
    def update(self, people, walls, passable_walls):
        if not self.active:
            return
            
        self.follow_path(path_waypoints)
        
        separation = np.zeros(2)
        for other in people:
            if other != self and other.active:
                dist = np.linalg.norm(self.position - other.position)
                if dist < SEPARATION_RADIUS:
                    separation += (self.position - other.position) / (dist + 0.001)
        
        if not self.is_in_passage(self.position):
            for wall in walls:
                dist_to_wall = self.distance_to_wall(wall)
                if dist_to_wall < WALL_MARGIN:
                    separation += (self.position - wall[:2]) / (dist_to_wall + 0.001)
        
        self.velocity += separation * 0.1
        speed = np.linalg.norm(self.velocity)
        if speed > MAX_SPEED:
            self.velocity = (self.velocity / speed) * MAX_SPEED
        self.position += self.velocity
        self.position = np.clip(self.position, [0, 0], [WIDTH, HEIGHT])
    
    def distance_to_wall(self, wall):
        x1, y1, x2, y2 = wall
        px, py = self.position
        line_length = np.linalg.norm(np.array([x2 - x1, y2 - y1]))
        if line_length == 0:
            return np.linalg.norm(self.position - np.array([x1, y1]))
        t = max(0, min(1, np.dot([px - x1, py - y1], [x2 - x1, y2 - y1]) / line_length**2))
        closest_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
        return np.linalg.norm(self.position - closest_point)
    
    def draw(self, screen):
        if self.active:
            pygame.draw.circle(screen, RED, self.position.astype(int), self.radius)

# Initialize empty people list
people = []
active_count = 0

# Path waypoints
path_waypoints = [
    np.array([200, 200]),
    np.array([1600, 200]),
    np.array([1600, 800]),
    np.array([200, 800])
]

# Walls
walls = [
    [0, 0, WIDTH, 0],
    [WIDTH, 0, WIDTH, HEIGHT],
    [WIDTH, HEIGHT, 0, HEIGHT],
    [0, HEIGHT, 0, 0],
    [0, 400, 800, 400],
    [1000, 400, WIDTH, 400],
    #[900, 400, 900, 0],
    [900, 600, 900, HEIGHT]
]

# Passable walls
passable_walls = [
    [800, 380, 1000, 420],
    [890, 400, 910, 600]
]

# Spawn timer
last_spawn_time = 0

running = True
while running:
    current_time = pygame.time.get_ticks()
    
    for event in pygame.event.get():
        if event.type == QUIT:
            running = False
    
    # Spawn new groups periodically
    if current_time - last_spawn_time > SPAWN_INTERVAL and active_count < NUM_PEOPLE:
        # Spawn a new group
        spawn_count = min(GROUP_SIZE, NUM_PEOPLE - active_count)
        for _ in range(spawn_count):
            if len(people) < NUM_PEOPLE:
                # Create new person
                x = random.randint(SPAWN_AREA[0], SPAWN_AREA[1])
                y = random.randint(SPAWN_AREA[2], SPAWN_AREA[3])
                people.append(Person(x, y))
            else:
                # Reactivate existing person
                for person in people:
                    if not person.active:
                        person.position = np.array([
                            random.randint(SPAWN_AREA[0], SPAWN_AREA[1]),
                            random.randint(SPAWN_AREA[2], SPAWN_AREA[3])
                        ], dtype=float)
                        person.velocity = np.random.rand(2) * MAX_SPEED
                        person.active = True
                        break
        active_count += spawn_count
        last_spawn_time = current_time
    
    screen.fill(WHITE)
    
    # Draw path
    for i, point in enumerate(path_waypoints):
        pygame.draw.circle(screen, GREEN, point.astype(int), 10)
        if i > 0:
            pygame.draw.line(screen, (200, 200, 200), path_waypoints[i-1], point, 2)
    
    # Draw solid walls
    for wall in walls:
        pygame.draw.line(screen, BLACK, (wall[0], wall[1]), (wall[2], wall[3]), 5)
    
    # Draw passable walls
    for wall in passable_walls:
        x1, y1, x2, y2 = wall
        rect = pygame.Rect(min(x1,x2), min(y1,y2), abs(x2-x1), abs(y2-y1))
        s = pygame.Surface((abs(x2-x1), abs(y2-y1)), pygame.SRCALPHA)
        s.fill(BLUE)
        screen.blit(s, (min(x1,x2), min(y1,y2)))
    
    # Update and draw people
    active_count = 0
    for person in people:
        person.update(people, walls, passable_walls)
        person.draw(screen)
        if person.active:
            active_count += 1
    
    # Display spawn info
    info_text = f"Active: {active_count}/{NUM_PEOPLE}  Next spawn in: {max(0, (SPAWN_INTERVAL - (current_time - last_spawn_time))//1000) }s"
    text_surface = font.render(info_text, True, BLACK)
    screen.blit(text_surface, (10, 10))
    
    pygame.display.flip()
    clock.tick(60)

pygame.quit()
sys.exit()

SystemExit: 

In [4]:
# Version 1.3

# This is just the program with a supermarket simulation
import pygame
import numpy as np
import sys
import random
from pygame.locals import *
import math

# Parameters
WIDTH, HEIGHT = 1800, 1000
NUM_PEOPLE = 50
PERSON_RADIUS = 5
MAX_SPEED = 1
SEPARATION_RADIUS = 20
WALL_MARGIN = 15 # Separación entre las paredes
SPAWN_AREA = (40, 150, 30, 80)  # x_min, x_max, y_min, y_max
GROUP_SIZE = 2                   # Number of people per group
SPAWN_INTERVAL = 3500            # Milliseconds between group spawns
INFECTION_RATE = 0.2  # 20% de las personas estarán infectadas

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)  # For path waypoints
BLUE = (0, 0, 255)   # For passable walls
YELLOW = (255, 255, 0)

# Pygame setup
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
font = pygame.font.SysFont('Arial', 20)

# Define the initial path waypoints 
'''These are the entries of the space that we are analyzing'''
initial_path_waypoints = [np.array([200, 100])]


# Define the final path waypoints 
'''These are the exits of the space that we are analyzing'''
final_path_waypoints = [np.array([350, 100]),
                        np.array([50, 100]),]

# Define path waypoints (green)
middle_path_waypoints = [# 975 is the midle point of the market
    #np.array([1625, 100]),
    #np.array([275, 100]),
    
    np.array([600, 100]),
    np.array([850, 100]),
    np.array([1100, 100]),
    np.array([1350, 100]),
    
    np.array([600, 200]),
    np.array([850, 200]),
    np.array([1100, 200]),
    np.array([1350, 200]),
    
    np.array([600, 300]),
    np.array([850, 300]),
    np.array([1100, 300]),
    np.array([1350, 300]),
    
    np.array([600, 400]),
    np.array([850, 400]),
    np.array([1100, 400]),
    np.array([1350, 400]),
    
    np.array([600, 500]),
    np.array([850, 500]),
    np.array([1100, 500]),
    np.array([1350, 500]),
    
    np.array([600, 600]),
    np.array([850, 600]),
    np.array([1100, 600]),
    np.array([1350, 600]),
    
    np.array([600, 700]),
    np.array([850, 700]),
    np.array([1100, 700]),
    np.array([1350, 700]),

]

# Cajeros
#middle_path_waypoints = [np.array([1600, 800]),
#                          np.array([300, 800])]

class Person:
    # Variables iniciales de la clase Person
    def __init__(self, x, y, path):
        self.position = np.array([x, y], dtype=float)
        self.velocity = np.random.rand(2) * MAX_SPEED
        self.radius = PERSON_RADIUS
        self.path_index = 0
        self.path_threshold = 20  # Distance to consider waypoint reached
        self.active = True
        self.path = path  # Each person gets its own path
        self.color = BLUE # Susceptible person
        
    
    def follow_path(self):
        if self.path_index < len(self.path):
            target = self.path[self.path_index]
            direction = target - self.position
            distance = np.linalg.norm(direction)
            
            if distance < self.path_threshold:
                self.path_index += 1
                # If reached final waypoint, deactivate
                if self.path_index >= len(self.path):
                    self.active = False
            elif distance > 0:
                direction_normalized = direction / distance
                self.velocity += direction_normalized * 0.1
    
    def update(self, people, walls, passable_walls):
        # Si no se encuentra activo la persona, que se devuelva
        if not self.active:
            return
        
        # Follow the path
        self.follow_path()
        
        # Vector para guardar la separación entre otras personas
        separation = np.zeros(2)
        # Se itera entre todas las instancias menos la que se está analizando
        for other in people:
            if other != self and other.active:
                dist = np.linalg.norm(self.position - other.position) # Se calcula la distancia
                if dist < SEPARATION_RADIUS: # Si la distancia de separación es menor que el límite permitido, se aleja
                    separation += (self.position - other.position) / (dist + 0.0001)  # Avoid division by zero
        
        # Wall avoidance (only for solid walls)
        # Se itera entre todas las paredes
        for wall in walls:
            dist_to_wall = self.distance_to_wall(wall)
            # Si la distancia es menor a la permitida se separa de la pared.
            if dist_to_wall < WALL_MARGIN:
                separation += (self.position - wall[:2]) / (dist_to_wall + 0.0001)
        
        # Pass through passable walls (no avoidance)
        # We could add special behavior here if needed
        
        # Update velocity and position
        self.velocity += separation * 0.1
        speed = np.linalg.norm(self.velocity)
        if speed > MAX_SPEED:
            self.velocity = (self.velocity / speed) * MAX_SPEED
        self.position += self.velocity
        
        # Keep within screen bounds
        self.position = np.clip(self.position, [0, 0], [WIDTH, HEIGHT])
    
    def distance_to_wall(self, wall):
        x1, y1, x2, y2 = wall
        px, py = self.position
        line_length = np.linalg.norm(np.array([x2 - x1, y2 - y1]))
        if line_length == 0:
            return np.linalg.norm(self.position - np.array([x1, y1]))
        t = max(0, min(1, np.dot([px - x1, py - y1], [x2 - x1, y2 - y1]) / line_length**2))
        closest_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
        return np.linalg.norm(self.position - closest_point)
    
    def draw(self, screen):
        if self.active:
            pygame.draw.circle(screen, self.color, self.position.astype(int), self.radius)

        

# Define walls (black - solid, must avoid)
walls = [# A vector is define like this [x1, y1, x2, y2]
    # Screen boundaries
    [0, 0, WIDTH, 0],        # Top wall
    [WIDTH, 0, WIDTH, HEIGHT],  # Right wall
    [WIDTH, HEIGHT, 0, HEIGHT],  # Bottom wall
    [0, HEIGHT, 0, 0],        # Left wall
    
    # Interior walls with passages
    [200, 50, 1700, 50], # top wall market
    [1700, 50, 1700, 900], # right wall market
    [1700, 900, 200, 900], # botton wall market
    [200, 900, 200, 150], # left wall market
    
    # Pasillos
    [400, 150, 1500, 150], # division 1
    [400, 250, 1500, 250], # division 2
    [400, 350, 1500, 350], # division 3
    [400, 450, 1500, 450], # division 4
    [400, 550, 1500, 550], # division 5
    [400, 650, 1500, 650], # division 6
    
    
    # Original walls
    #[0, 400, 800, 400],      # Horizontal wall left
    #[1000, 400, WIDTH - 150, 400], # Horizontal wall right (gap between 800-1000)
    
    #[900, 400, 900, 0],      # Vertical wall top
    #[900, 600, 900, HEIGHT]  # Vertical wall bottom (gap between 400-600)
]

# Define passable walls (blue - can walk through)
passable_walls = [
#    [800, 400, 1000, 400],   # Passage between horizontal walls
#    [900, 400, 900, 600]     # Passage between vertical walls
]

# Initialize empty people list
people = []
active_count = 0
# Spawn timer
last_spawn_time = 0
current_group = 0

midle_value = 975 # half of the mall

# Create function to generate path way 
def generate_random_path():
    """Generate a path with random middle waypoints (3-8) between fixed start/end"""
    # Randomly select 3-8 waypoints from the middle path
    num_waypoints = random.randint(3, 8)
    selected_waypoints = random.sample(middle_path_waypoints, num_waypoints)
    
    # Combine with fixed start and end points
    full_path = selected_waypoints + final_path_waypoints
    
    # We intialize the whole path way
    final_path = initial_path_waypoints
        
    # Make a loop to make the path way smoother for the agente
    i = 0
    for point in full_path:
        # Select each value of the actual objective
        row = point[0]
        column = point[1]
        
        # We calculate the exit of the person
        if final_path[-1][0] > midle_value:
            final_path = final_path +  [np.array([1625, final_path[-1][1]])]
        else:
            final_path = final_path + [np.array([275, final_path[-1][1]])]
        
        # We calcule the entry to the hallway
        # The row value is the last value of the list final_path
        final_path = final_path + [np.array([final_path[-1][0], column])]
        
        # We add the actual objective
        final_path = final_path + [point]
        
        # We iterate to next objective        
        i = i + 1
    
    return final_path

# Main loop
running = True
while running:
    # Obtenemos el tiempo actual de la simulación
    current_time = pygame.time.get_ticks()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
    # Aparecen nuevos grupos de manera periodica
    if current_time - last_spawn_time > SPAWN_INTERVAL and active_count < NUM_PEOPLE:
        # Generar camino
        group_path = generate_random_path()
        # Generar grupo
        spawn_count = min(GROUP_SIZE, NUM_PEOPLE - active_count)
        for _ in range(spawn_count):
            if len(people) < NUM_PEOPLE:
                # Create new person
                x = random.randint(SPAWN_AREA[0], SPAWN_AREA[1])
                y = random.randint(SPAWN_AREA[2], SPAWN_AREA[3])
                new_person = Person(x, y, group_path)
                people.append(new_person)
            else:
                # Reactivate existing person
                for person in people:
                    if not person.active:
                        person.position = np.array([
                            random.randint(SPAWN_AREA[0], SPAWN_AREA[1]),
                            random.randint(SPAWN_AREA[2], SPAWN_AREA[3])
                        ], dtype=float)
                        person.velocity = np.random.rand(2) * MAX_SPEED
                        person.path = group_path
                        person.path_index = 0
                        person.active = True
                        break
        active_count += spawn_count
        last_spawn_time = current_time
        current_group += 1
    
    screen.fill(WHITE)
    
    # Draw path waypoints
    for i, point in enumerate(middle_path_waypoints):
        pygame.draw.circle(screen, GREEN, point.astype(int), 10)
        if i > 0:
            pygame.draw.line(screen, (200, 200, 200), 
                           middle_path_waypoints[i-1], point, 2)
    
    # Draw walls (solid)
    for wall in walls:
        pygame.draw.line(screen, BLACK, (wall[0], wall[1]), (wall[2], wall[3]), 5)
    
    # Draw passable walls (can walk through)
    for wall in passable_walls:
        pygame.draw.line(screen, RED, (wall[0], wall[1]), (wall[2], wall[3]), 3)
    
    # Update and draw people
    active_count = 0
    for person in people:
        person.update(people, walls, passable_walls)
        person.draw(screen)
        if person.active:
            active_count += 1
            
    
    # Display spawn info
    info_text = f"Active: {active_count}/{NUM_PEOPLE}  Next spawn in: {max(0, (SPAWN_INTERVAL - (current_time - last_spawn_time))//1000) }s"
    text_surface = font.render(info_text, True, BLACK)
    screen.blit(text_surface, (10, 10))
    
    pygame.display.flip()
    clock.tick(60)
    
pygame.quit()
sys.exit()

SystemExit: 

In [1]:
# Version 1.4

# In this version a the velocity was change, the respawn area and now exist a radius of infection
import pygame
import numpy as np
import sys
import random
from pygame.locals import *
import math

# Parameters
WIDTH, HEIGHT = 1800, 1000
NUM_PEOPLE = 50 # This should be 50
PERSON_RADIUS = 5
MAX_SPEED = 0.463 
SEPARATION_RADIUS = 20
WALL_MARGIN = 15 # Separación entre las paredes
SPAWN_AREA = (40, 100, 900, 930)  # x_min, x_max, y_min, y_max
GROUP_SIZE = 2                    # Number of people per group
SPAWN_INTERVAL = 3500            # Milliseconds between group spawns
INFECTION_RATE = 0.1  # % de las personas estarán infectadas
ASYMPTOMATIC_RATE = 0.01  # % de las personas serán asintómaticas

# Aleatoriedad de caminos, cantidad de middle path waypoint que una persona puede tener en un recorrido.
CASO_MENOR = 1
CASO_MAYOR = 3

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)  # For path waypoints
BLUE = (0, 0, 255)   # For passable walls
YELLOW = (255, 255, 0)

# Pygame setup
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
font = pygame.font.SysFont('Arial', 20)

# Define the initial path waypoints 
'''These are the entries of the space that we are analyzing'''
initial_path_waypoints = [
    np.array([100, 800]),
    np.array([100, 100]),
    ]

# Define the final path waypoints 
'''These are the exits of the space that we are analyzing'''
final_path_waypoints = [np.array([350, 100]),
                        np.array([50, 100]),]

# Define path waypoints (green)
middle_path_waypoints = [# 975 is the midle point of the market
    #np.array([1625, 100]),
    #np.array([275, 100]),
    
    np.array([600, 100]),
    np.array([850, 100]),
    np.array([1100, 100]),
    np.array([1350, 100]),
    
    np.array([600, 200]),
    np.array([850, 200]),
    np.array([1100, 200]),
    np.array([1350, 200]),
    
    np.array([600, 300]),
    np.array([850, 300]),
    np.array([1100, 300]),
    np.array([1350, 300]),
    
    np.array([600, 400]),
    np.array([850, 400]),
    np.array([1100, 400]),
    np.array([1350, 400]),
    
    np.array([600, 500]),
    np.array([850, 500]),
    np.array([1100, 500]),
    np.array([1350, 500]),
    
    np.array([600, 600]),
    np.array([850, 600]),
    np.array([1100, 600]),
    np.array([1350, 600]),
    
    np.array([600, 700]),
    np.array([850, 700]),
    np.array([1100, 700]),
    np.array([1350, 700]),

]

# Pegamos toda la lista de puntos para dibujar en el mapa las trayectorias 
path_waypoints = initial_path_waypoints + middle_path_waypoints + final_path_waypoints

# Cajeros
#middle_path_waypoints = [np.array([1600, 800]),
#                          np.array([300, 800])]

class Person:
    # Variables iniciales de la clase Person
    def __init__(self, x, y, path, infected=False, asymptomatic = False):
        self.position = np.array([x, y], dtype=float)
        self.velocity = np.random.rand(2) * MAX_SPEED
        self.radius = PERSON_RADIUS
        self.path_index = 0
        self.path_threshold = 20  # Distance to consider waypoint reached
        self.active = True
        self.path = path  # Each person gets its own path
        self.base_color = RED if infected else BLUE
        self.pulse_phase = 0  # Para el efecto de pulso
        
        #self.color = BLUE # Susceptible person
        self.infected = infected
        self.asymptomatic = asymptomatic
        if self.infected:
            self.base_color = RED
        elif self.asymptomatic:
            self.base_color = YELLOW
        else:
            self.base_color = BLUE
            
        
    
    def follow_path(self):
        if self.path_index < len(self.path):
            target = self.path[self.path_index]
            direction = target - self.position
            distance = np.linalg.norm(direction)
            
            if distance < self.path_threshold:
                self.path_index += 1
                # If reached final waypoint, deactivate
                if self.path_index >= len(self.path):
                    self.active = False
            elif distance > 0:
                direction_normalized = direction / distance
                self.velocity += direction_normalized * 0.1
    
    def update(self, people, walls, passable_walls):
        # Si no se encuentra activo la persona, que se devuelva
        if not self.active:
            return
        
        # Follow the path
        self.follow_path()
        
        # Vector para guardar la separación entre otras personas
        separation = np.zeros(2)
        # Se itera entre todas las instancias menos la que se está analizando
        for other in people:
            if other != self and other.active:
                dist = np.linalg.norm(self.position - other.position) # Se calcula la distancia
                if dist < SEPARATION_RADIUS: # Si la distancia de separación es menor que el límite permitido, se aleja
                    separation += (self.position - other.position) / (dist + 0.0001)  # Avoid division by zero
        
        # Wall avoidance (only for solid walls)
        # Se itera entre todas las paredes
        for wall in walls:
            dist_to_wall = self.distance_to_wall(wall)
            # Si la distancia es menor a la permitida se separa de la pared.
            if dist_to_wall < WALL_MARGIN:
                separation += (self.position - wall[:2]) / (dist_to_wall + 0.0001)
        
        # Pass through passable walls (no avoidance)
        # We could add special behavior here if needed
        
        # Update velocity and position
        self.velocity += separation * 0.1
        speed = np.linalg.norm(self.velocity)
        if speed > MAX_SPEED:
            self.velocity = (self.velocity / speed) * MAX_SPEED
        self.position += self.velocity
        
        # Keep within screen bounds
        self.position = np.clip(self.position, [0, 0], [WIDTH, HEIGHT])
    
    def distance_to_wall(self, wall):
        x1, y1, x2, y2 = wall
        px, py = self.position
        line_length = np.linalg.norm(np.array([x2 - x1, y2 - y1]))
        if line_length == 0:
            return np.linalg.norm(self.position - np.array([x1, y1]))
        t = max(0, min(1, np.dot([px - x1, py - y1], [x2 - x1, y2 - y1]) / line_length**2))
        closest_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
        return np.linalg.norm(self.position - closest_point)
    
    def draw(self, screen):
        if not self.active:
            return

        if self.infected or self.asymptomatic:
            # Tiempo animado en segundos
            time = pygame.time.get_ticks() / 250.0  # más suave que *0.005

            # Pulso de tamaño usando seno (entre 2.5x y 3.5x del radio)
            pulse_scale = 3 + 3 * math.sin(time)
            outer_radius = max(1, int(self.radius * pulse_scale))

            # Color pulsante (entre rosa claro y rojo intenso)
            pulse_color = (
                255,
                int(100 + 50 * math.sin(time)),  # valor entre 50 y 150
                int(100 + 50 * math.sin(time))
            )

            # Dibuja círculo externo
            pygame.draw.circle(screen, pulse_color, self.position, outer_radius, width=2)

        # Dibuja la persona (círculo base)
        pygame.draw.circle(screen, self.base_color, self.position, self.radius)

# Define walls (black - solid, must avoid)
walls = [# A vector is define like this [x1, y1, x2, y2]
    # Screen boundaries
    [0, 0, WIDTH, 0],        # Top wall
    [WIDTH, 0, WIDTH, HEIGHT],  # Right wall
    [WIDTH, HEIGHT, 0, HEIGHT],  # Bottom wall
    [0, HEIGHT, 0, 0],        # Left wall
    
    # Interior walls with passages
    [200, 50, 1700, 50], # top wall market
    [1700, 50, 1700, 900], # right wall market
    [1700, 900, 200, 900], # botton wall market
    [200, 900, 200, 150], # left wall market
    
    # Pasillos
    [400, 150, 1500, 150], # division 1
    [400, 250, 1500, 250], # division 2
    [400, 350, 1500, 350], # division 3
    [400, 450, 1500, 450], # division 4
    [400, 550, 1500, 550], # division 5
    [400, 650, 1500, 650], # division 6
    
    
    # Original walls
    #[0, 400, 800, 400],      # Horizontal wall left
    #[1000, 400, WIDTH - 150, 400], # Horizontal wall right (gap between 800-1000)
    
    #[900, 400, 900, 0],      # Vertical wall top
    #[900, 600, 900, HEIGHT]  # Vertical wall bottom (gap between 400-600)
]

# Define passable walls (blue - can walk through)
passable_walls = [
    #[800, 400, 1000, 400],   # Passage between horizontal walls
    #[900, 400, 900, 600]     # Passage between vertical walls
]

# Initialize empty people list
people = []
active_count = 0
# Spawn timer
last_spawn_time = 0
current_group = 0

midle_value = 975 # half of the mall

# Create function to generate path way 
def generate_random_path():
    """Generate a path with random middle waypoints (3-8) between fixed start/end"""
    # Randomly select 3-8 waypoints from the middle path
    num_waypoints = random.randint(CASO_MENOR, CASO_MAYOR)
    selected_waypoints = random.sample(middle_path_waypoints, num_waypoints)
    
    # Combine with fixed start and end points
    full_path = selected_waypoints + final_path_waypoints
    
    # We intialize the whole path way
    final_path = initial_path_waypoints
        
    # Make a loop to make the path way smoother for the agente
    i = 0
    for point in full_path:
        # Select each value of the actual objective
        row = point[0]
        column = point[1]
        
        # We calculate the exit of the person
        if final_path[-1][0] > midle_value:
            final_path = final_path +  [np.array([1625, final_path[-1][1]])]
        else:
            final_path = final_path + [np.array([275, final_path[-1][1]])]
        
        # We calcule the entry to the hallway
        # The row value is the last value of the list final_path
        final_path = final_path + [np.array([final_path[-1][0], column])]
        
        # We add the actual objective
        final_path = final_path + [point]
        
        # We iterate to next objective        
        i = i + 1
    
    return final_path

# Main loop
running = True
while running:
    # Obtenemos el tiempo actual de la simulación
    current_time = pygame.time.get_ticks()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
    # Aparecen nuevos grupos de manera periodica
    if current_time - last_spawn_time > SPAWN_INTERVAL and active_count < NUM_PEOPLE:
        # Generar camino
        group_path = generate_random_path()
        # Generar grupo
        spawn_count = min(GROUP_SIZE, NUM_PEOPLE - active_count)
        for _ in range(spawn_count):
            # Agregamos un efecto random de contagio al grupo
            infected = random.random() < INFECTION_RATE
            asymptomatic = False # Por default asymmptomatic esta en falso
            # Entre un grupo de personas infectadas está la posibilidad de ser asimtomatico
            if infected:
                asymptomatic = random.random() < ASYMPTOMATIC_RATE 
                
            if len(people) < NUM_PEOPLE:
                # Crear nueva persona por grupo
                x = random.randint(SPAWN_AREA[0], SPAWN_AREA[1])
                y = random.randint(SPAWN_AREA[2], SPAWN_AREA[3])
                new_person = Person(x, y, group_path, infected, asymptomatic)
                people.append(new_person)
            else:
                # Reactivate existing person
                for person in people:
                    if not person.active:
                        #person.position = np.array([
                        #    random.randint(SPAWN_AREA[0], SPAWN_AREA[1]),
                        #    random.randint(SPAWN_AREA[2], SPAWN_AREA[3])
                        #], dtype=float)
                        #person.velocity = np.random.rand(2) * MAX_SPEED
                        #person.path = group_path
                        #person.path_index = 0
                        person.active = True
                        #person.infected = infected
                        #person.asymptomatic = asymptomatic
                        #person.infected = infected
                        
                        #person.asymptomatic = asymptomatic
                        #if person.infected:
                        #    person.base_color = RED
                        #elif person.asymptomatic:
                        #    person.base_color = YELLOW
                        #else:
                        #    person.base_color = BLUE
                        break
        active_count += spawn_count
        last_spawn_time = current_time
        current_group += 1
    
    screen.fill(WHITE)
    
    # Draw path waypoints
    for i, point in enumerate(path_waypoints):
        pygame.draw.circle(screen, GREEN, point.astype(int), 10)
        if i > 0:
            pygame.draw.line(screen, (200, 200, 200), 
                           path_waypoints[i-1], point, 2)
    
    # Draw walls (solid)
    for wall in walls:
        pygame.draw.line(screen, BLACK, (wall[0], wall[1]), (wall[2], wall[3]), 5)
    
    # Draw passable walls (can walk through)
    for wall in passable_walls:
        pygame.draw.line(screen, RED, (wall[0], wall[1]), (wall[2], wall[3]), 3)
    
    # Update and draw people
    active_count = 0
    for person in people:
        person.update(people, walls, passable_walls)
        person.draw(screen)
        if person.active:
            active_count += 1
            
    
    # Display spawn info
    info_text = f"Active: {active_count}/{NUM_PEOPLE}  Next spawn in: {max(0, (SPAWN_INTERVAL - (current_time - last_spawn_time))//1000) }s"
    text_surface = font.render(info_text, True, BLACK)
    screen.blit(text_surface, (10, 10))
    
    pygame.display.flip()
    clock.tick(60)
    
pygame.quit()
sys.exit()

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

/home/nelson/Desktop/master/tfm/code/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# Version 1.5

# Checkout points add and timer add
import pygame
import numpy as np
import sys
import random
from pygame.locals import *
import math

# Parameters
WIDTH, HEIGHT = 1800, 1000
NUM_PEOPLE = 50 # This should be 50
PERSON_RADIUS = 5
MAX_SPEED = 2#0.463 
SEPARATION_RADIUS = 20
WALL_MARGIN = 15 # Separación entre las paredes
SPAWN_AREA = (40, 100, 900, 930)  # x_min, x_max, y_min, y_max
GROUP_SIZE = 1                    # Number of people per group
SPAWN_INTERVAL = 3500            # Milliseconds between group spawns
INFECTION_RATE = 0.1  # % de las personas estarán infectadas
ASYMPTOMATIC_RATE = 0.01  # % de las personas serán asintómaticas

# Aleatoriedad de caminos, cantidad de middle path waypoint que una persona puede tener en un recorrido.
CASO_MENOR = 1
CASO_MAYOR = 3

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)  # For path waypoints
BLUE = (0, 0, 255)   # For passable walls
YELLOW = (255, 255, 0)

# Pygame setup
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
font = pygame.font.SysFont('Arial', 20)

# Define the initial path waypoints 
'''These are the entries of the space that we are analyzing'''
initial_path_waypoints = [
    np.array([100, 800]),
    np.array([100, 100]),
    ]

# Define the final path waypoints 
'''These are the exits of the space that we are analyzing'''
final_path_waypoints = [
    np.array([350, 600]),
    np.array([350, 650]),
    ]

# Define path waypoints (green)
middle_path_waypoints = [# 975 is the midle point of the market
    np.array([600, 100]),
    np.array([850, 100]),
    np.array([1100, 100]),
    np.array([1350, 100]),
    
    np.array([600, 200]),
    np.array([850, 200]),
    np.array([1100, 200]),
    np.array([1350, 200]),

    np.array([600, 300]),
    np.array([850, 300]),
    np.array([1100, 300]),
    np.array([1350, 300]),

    np.array([600, 400]),
    np.array([850, 400]),
    np.array([1100, 400]),
    np.array([1350, 400]),

    np.array([600, 500]),
    np.array([850, 500]),
    np.array([1100, 500]),
    np.array([1350, 500]),

    np.array([600, 600]),
    np.array([850, 600]),
    np.array([1100, 600]),
    np.array([1350, 600]),
]

checkout_path_waypoints = [
    np.array([275, 100]), 
    
    # Fila 1
    np.array([350, 700]), 
    np.array([350, 750]), 
    np.array([350, 800]), 
    np.array([350, 850]),
    
    np.array([600, 700]), 
    np.array([600, 750]), 
    np.array([600, 800]),
    np.array([600, 850]), 
    
    np.array([850, 700]),
    np.array([850, 750]),
    np.array([850, 800]),
    np.array([850, 850]),
    
    np.array([1100, 700]),
    np.array([1100, 750]),
    np.array([1100, 800]),
    np.array([1100, 850]),
    
    np.array([1350, 700]),
    np.array([1350, 750]),
    np.array([1350, 800]),
    np.array([1350, 850]),
    
    np.array([1600, 700]),
    np.array([1600, 750]),
    np.array([1600, 800]),
    np.array([1600, 850]),
     
]

# Pegamos toda la lista de puntos para dibujar en el mapa las trayectorias 
path_waypoints = initial_path_waypoints + middle_path_waypoints + final_path_waypoints

class Person:
    # Variables iniciales de la clase Person
    def __init__(self, x, y, path, infected=False, asymptomatic = False):
        # Add waiting-related attributes
        self.waiting = False
        self.wait_time = random.uniform(0.5, 2.0)  # Wait between 0.5-2 seconds
        self.wait_start_time = 0

        self.position = np.array([x, y], dtype=float)
        self.velocity = np.random.rand(2) * MAX_SPEED
        self.radius = PERSON_RADIUS
        self.path_index = 0
        self.path_threshold = 20  # Distance to consider waypoint reached
        self.active = True
        self.path = path  # Each person gets its own path
        self.base_color = RED if infected else BLUE
        self.pulse_phase = 0  # Para el efecto de pulso
        
        #self.color = BLUE # Susceptible person
        self.infected = infected
        self.asymptomatic = asymptomatic
        if self.infected:
            self.base_color = RED
        elif self.asymptomatic:
            self.base_color = YELLOW
        else:
            self.base_color = BLUE
            
    def follow_path(self):
        if self.path_index < len(self.path):
            target = self.path[self.path_index]
            direction = target - self.position
            distance = np.linalg.norm(direction)
            
            # Definir los puntos donde NO debe esperar
            x_value = self.path[self.path_index][0]
            is_special_point = x_value in [275, 1625]
            is_first_or_second = self.path_index in [0, 1]
            is_last = self.path_index == len(self.path) - 1

            if distance < self.path_threshold:
                # Solo espera en waypoints intermedios, que NO son especiales
                if (
                    2 < self.path_index < len(self.path) - 1 and  # waypoints intermedios
                    not self.waiting and
                    not is_special_point
                ):
                    # Iniciar espera
                    self.waiting = True
                    self.wait_start_time = pygame.time.get_ticks()
                    self.velocity = np.zeros(2)  # Detenerse
                elif self.waiting:
                    # Checar si terminó de esperar
                    if pygame.time.get_ticks() - self.wait_start_time > self.wait_time * 1000:
                        self.waiting = False
                        self.path_index += 1
                else:
                    # Para los puntos especiales, los dos primeros y el último: avanzar sin esperar
                    self.path_index += 1

                # Si llegó al final, desactiva el agente
                if self.path_index >= len(self.path):
                    self.active = False
            elif distance > 0 and not self.waiting:
                direction_normalized = direction / distance
                self.velocity += direction_normalized * 0.1

    def update(self, people, walls):
        if not self.active:
            return
        
        # Follow the path
        self.follow_path()
        
        if self.waiting:
            return  # Skip movement calculations while waiting
            
        # Rest of the update method remains the same...
        separation = np.zeros(2)
        for other in people:
            if other != self and other.active:
                dist = np.linalg.norm(self.position - other.position)
                if dist < SEPARATION_RADIUS:
                    separation += (self.position - other.position) / (dist + 0.0001)
        
        for wall in walls:
            dist_to_wall = self.distance_to_wall(wall)
            if dist_to_wall < WALL_MARGIN:
                separation += (self.position - wall[:2]) / (dist_to_wall + 0.0001)
        
        self.velocity += separation * 0.1
        speed = np.linalg.norm(self.velocity)
        if speed > MAX_SPEED:
            self.velocity = (self.velocity / speed) * MAX_SPEED
        self.position += self.velocity
        self.position = np.clip(self.position, [0, 0], [WIDTH, HEIGHT])
    
    def distance_to_wall(self, wall):
        x1, y1, x2, y2 = wall
        px, py = self.position
        line_length = np.linalg.norm(np.array([x2 - x1, y2 - y1]))
        if line_length == 0:
            return np.linalg.norm(self.position - np.array([x1, y1]))
        t = max(0, min(1, np.dot([px - x1, py - y1], [x2 - x1, y2 - y1]) / line_length**2))
        closest_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
        return np.linalg.norm(self.position - closest_point)
    
    def draw(self, screen):
        if not self.active:
            return

        if self.infected or self.asymptomatic:
            # Tiempo animado en segundos
            time = pygame.time.get_ticks() / 250.0  # más suave que *0.005

            # Pulso de tamaño usando seno (entre 2.5x y 3.5x del radio)
            pulse_scale = 3 + 3 * math.sin(time)
            outer_radius = max(1, int(self.radius * pulse_scale))

            # Color pulsante (entre rosa claro y rojo intenso)
            pulse_color = (
                255,
                int(100 + 50 * math.sin(time)),  # valor entre 50 y 150
                int(100 + 50 * math.sin(time))
            )

            # Dibuja círculo externo
            pygame.draw.circle(screen, pulse_color, self.position, outer_radius, width=2)

        # Dibuja la persona (círculo base)
        pygame.draw.circle(screen, self.base_color, self.position, self.radius)

# Define walls (black - solid, must avoid)
walls = [# A vector is define like this [x1, y1, x2, y2]
    # Screen boundaries
    [0, 0, WIDTH, 0],        # Top wall
    [WIDTH, 0, WIDTH, HEIGHT],  # Right wall
    [WIDTH, HEIGHT, 0, HEIGHT],  # Bottom wall
    [0, HEIGHT, 0, 0],        # Left wall
    
    # Interior walls with passages
    [200, 50, 1700, 50], # top wall market
    [1700, 50, 1700, 900], # right wall market
    [1700, 900, 200, 900], # botton wall market
    [200, 900, 200, 150], # left wall market
    
    # Pasillos
    [400, 150, 1500, 150], # division 1
    [400, 250, 1500, 250], # division 2
    [400, 350, 1500, 350], # division 3
    [400, 450, 1500, 450], # division 4
    [400, 550, 1500, 550], # division 5
    #[400, 650, 1500, 650], # division 6
]

# Define passable walls (blue - can walk through)
passable_walls = [
    #[800, 400, 1000, 400],   # Passage between horizontal walls
    #[900, 400, 900, 600]     # Passage between vertical walls
]

# Initialize empty people list
people = []
active_count = 0
# Spawn timer
last_spawn_time = 0
current_group = 0

midle_value = 975 # half of the mall

# Create function to generate path way 
def generate_random_path():
    """Generate a path with random middle waypoints (3-8) between fixed start/end"""
    # Randomly select 3-8 waypoints from the middle path
    num_waypoints = random.randint(CASO_MENOR, CASO_MAYOR)
    selected_waypoints = random.sample(middle_path_waypoints, num_waypoints)
    
    # Combine with fixed start and end points, points of interest and exit
    full_path = selected_waypoints + final_path_waypoints
    
    # Intialize the whole path way
    final_path = initial_path_waypoints
        
    # Make a loop to make the path way smoother for the agente
    i = 0
    for point in full_path:
        # Select each value of the actual objective
        row = point[0]
        column = point[1]
        
        # We calculate the exit of the person in the hallway
        if final_path[-1][0] > midle_value:
            final_path = final_path +  [np.array([1625, final_path[-1][1]])]
        else:
            final_path = final_path + [np.array([275, final_path[-1][1]])]
        
        # We calcule the entry to the hallway
        # The row value is the last value of the list final_path
        final_path = final_path + [np.array([final_path[-1][0], column])]
        
        # We add the actual objective
        final_path = final_path + [point]
        
        # We iterate to next objective        
        i = i + 1
    
    return final_path

# Main loop
running = True
while running:
    # Obtenemos el tiempo actual de la simulación
    current_time = pygame.time.get_ticks()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
    # Aparecen nuevos grupos de manera periodica
    if current_time - last_spawn_time > SPAWN_INTERVAL and active_count < NUM_PEOPLE:
        # Generar camino
        group_path = generate_random_path()
        # Generar grupo
        spawn_count = min(GROUP_SIZE, NUM_PEOPLE - active_count)
        for _ in range(spawn_count):
            # Agregamos un efecto random de contagio al grupo
            infected = random.random() < INFECTION_RATE
            asymptomatic = False # Por default asymmptomatic esta en falso
            # Entre un grupo de personas infectadas está la posibilidad de ser asimtomatico
            if infected:
                asymptomatic = random.random() < ASYMPTOMATIC_RATE 
                
            if len(people) < NUM_PEOPLE:
                # Crear nueva persona por grupo
                x = random.randint(SPAWN_AREA[0], SPAWN_AREA[1])
                y = random.randint(SPAWN_AREA[2], SPAWN_AREA[3])
                new_person = Person(x, y, group_path, infected, asymptomatic)
                people.append(new_person)
            else:
                # Reactivate existing person
                for person in people:
                    if not person.active:
                        #person.position = np.array([
                        #    random.randint(SPAWN_AREA[0], SPAWN_AREA[1]),
                        #    random.randint(SPAWN_AREA[2], SPAWN_AREA[3])
                        #], dtype=float)
                        #person.velocity = np.random.rand(2) * MAX_SPEED
                        #person.path = group_path
                        #person.path_index = 0
                        person.active = True
                        #person.infected = infected
                        #person.asymptomatic = asymptomatic
                        #person.infected = infected
                        
                        #person.asymptomatic = asymptomatic
                        #if person.infected:
                        #    person.base_color = RED
                        #elif person.asymptomatic:
                        #    person.base_color = YELLOW
                        #else:
                        #    person.base_color = BLUE
                        break
        active_count += spawn_count
        last_spawn_time = current_time
        current_group += 1
    
    screen.fill(WHITE)
    
    # Draw path waypoints
    for i, point in enumerate(path_waypoints):
        pygame.draw.circle(screen, GREEN, point.astype(int), 10)
        if i > 0:
            pygame.draw.line(screen, (200, 200, 200), 
                           path_waypoints[i-1], point, 2)
            
    for i, point in enumerate(checkout_path_waypoints):
        pygame.draw.circle(screen, YELLOW, point.astype(int), 10)
    
    # Draw walls (solid)
    for wall in walls:
        pygame.draw.line(screen, BLACK, (wall[0], wall[1]), (wall[2], wall[3]), 5)
    
    # Draw passable walls (can walk through)
    for wall in passable_walls:
        pygame.draw.line(screen, RED, (wall[0], wall[1]), (wall[2], wall[3]), 3)
    
    # Update and draw people
    active_count = 0
    for person in people:
        person.update(people, walls)
        person.draw(screen)
        if person.active:
            active_count += 1
            
    
    # Display spawn info
    info_text = f"Active: {active_count}/{NUM_PEOPLE}  Next spawn in: {max(0, (SPAWN_INTERVAL - (current_time - last_spawn_time))//1000) }s"
    text_surface = font.render(info_text, True, BLACK)
    screen.blit(text_surface, (10, 10))
    
    pygame.display.flip()
    clock.tick(60)
    
pygame.quit()
sys.exit()

In [ ]:
# Version 1.6 checkout add
# Version 1.4

# In this version a the velocity was change, the respawn area and now exist a radius of infection
import pygame
import numpy as np
import sys
import random
from pygame.locals import *
import math

# Parameters
WIDTH, HEIGHT = 1800, 1000
NUM_PEOPLE = 75 # This should be 50
PERSON_RADIUS = 5
MAX_SPEED = 2 #0.463 t
SEPARATION_RADIUS = 15
WALL_MARGIN = 15 # Separación entre las paredes
SPAWN_AREA = (40, 100, 900, 930)  # x_min, x_max, y_min, y_max
GROUP_SIZE = 1                    # Number of people per group
SPAWN_INTERVAL = 500 #3500            # Milliseconds between group spawns
INFECTION_RATE = 0.1  # % de las personas estarán infectadas
ASYMPTOMATIC_RATE = 0.1  # % de las personas serán asintómaticas

# Aleatoriedad de caminos, cantidad de middle path waypoint que una persona puede tener en un recorrido.
CASO_MENOR = 1
CASO_MAYOR = 2

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)  # For path waypoints
BLUE = (0, 0, 255)   # For passable walls
YELLOW = (255, 255, 0)

# Wait times
WALK_WAIT_RANGE = (0.5, 8.0)      # seconds
CHECKOUT_WAIT_RANGE = (4.0, 8.0) # seconds  ← tweak this

# Pygame setup
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
font = pygame.font.SysFont('Arial', 20)

# Define the initial path waypoints 
'''These are the entries of the space that we are analyzing'''
initial_path_waypoints = [
    np.array([100, 800]),
    np.array([100, 100]),
]

# Define the final path waypoints 
'''These are the exits of the space that we are analyzing'''
final_path_waypoints = [
    np.array([350, 600]),
    np.array([700, 600]),
    np.array([1100, 600]),
    np.array([1600, 600]),
    ]

# Define path waypoints (green)
middle_path_waypoints = [# 975 is the midle point of the market
    np.array([600, 100]),
    np.array([850, 100]),
    np.array([1100, 100]),
    np.array([1350, 100]),
    
    np.array([600, 200]),
    np.array([850, 200]),
    np.array([1100, 200]),
    np.array([1350, 200]),

    np.array([600, 300]),
    np.array([850, 300]),
    np.array([1100, 300]),
    np.array([1350, 300]),

    np.array([600, 400]),
    np.array([850, 400]),
    np.array([1100, 400]),
    np.array([1350, 400]),

    np.array([600, 500]),
    np.array([850, 500]),
    np.array([1100, 500]),
    np.array([1350, 500]),

    np.array([600, 600]),
    np.array([850, 600]),
    np.array([1100, 600]),
    np.array([1350, 600]),
]

checkout_map = {
    "F1": [
        [350, 650, -1], 
        [350, 700, -1], 
        [350, 750, -1],        
        [350, 800, -1],        
        [350, 850, -1],        
    ],
    "F2": [
        [470, 650, -1], 
        [470, 700, -1], 
        [470, 750, -1],        
        [470, 800, -1],        
        [470, 850, -1],        
    ],
    "F3": [
        [600, 650, -1], 
        [600, 700, -1], 
        [600, 750, -1],        
        [600, 800, -1],        
        [600, 850, -1],        
    ],
    "F4": [
        [720, 650, -1], 
        [720, 700, -1], 
        [720, 750, -1],        
        [720, 800, -1],        
        [720, 850, -1],        
    ],
    "F5": [
        [850, 650, -1], 
        [850, 700, -1], 
        [850, 750, -1],        
        [850, 800, -1],        
        [850, 850, -1],        
    ],
    "F6": [
        [970, 650, -1], 
        [970, 700, -1], 
        [970, 750, -1],        
        [970, 800, -1],        
        [970, 850, -1],        
    ],
    "F7": [
        [1100, 650, -1], 
        [1100, 700, -1], 
        [1100, 750, -1],        
        [1100, 800, -1],        
        [1100, 850, -1],        
    ],
    "F8": [
        [1220, 650, -1], 
        [1220, 700, -1], 
        [1220, 750, -1],        
        [1220, 800, -1],        
        [1220, 850, -1],        
    ],
    "F9": [
        [1350, 650, -1], 
        [1350, 700, -1], 
        [1350, 750, -1],        
        [1350, 800, -1],        
        [1350, 850, -1],        
    ],
    "F10": [
        [1470, 650, -1], 
        [1470, 700, -1], 
        [1470, 750, -1],        
        [1470, 800, -1],        
        [1470, 850, -1],        
    ],
    "F11": [
        [1600, 650, -1], 
        [1600, 700, -1], 
        [1600, 750, -1],        
        [1600, 800, -1],        
        [1600, 850, -1],        
    ],
    
}

# Define walls (black - solid, must avoid)
walls = [# A vector is define like this [x1, y1, x2, y2]
    # Screen boundaries
    [0, 0, WIDTH, 0],        # Top wall
    [WIDTH, 0, WIDTH, HEIGHT],  # Right wall
    [WIDTH, HEIGHT, 0, HEIGHT],  # Bottom wall
    [0, HEIGHT, 0, 0],        # Left wall
    
    # Interior walls with passages
    [200, 50, 1700, 50], # top wall market
    [1700, 50, 1700, 900], # right wall market
    [1550, 900, 200, 900], # botton wall market
    [200, 900, 200, 150], # left wall market
    
    # Pasillos
    [400, 150, 1500, 150], # division 1
    [400, 250, 1500, 250], # division 2
    [400, 350, 1500, 350], # division 3
    [400, 450, 1500, 450], # division 4
    [400, 550, 1500, 550], # division 5
]

# Functions

# Create function to generate path way 
def generate_random_path():
    """Generate a path with random middle waypoints (3-8) between fixed start/end"""
    # Randomly select 3-8 waypoints from the middle path
    num_waypoints = random.randint(CASO_MENOR, CASO_MAYOR)
    selected_waypoints = random.sample(middle_path_waypoints, num_waypoints)
    
    # Combine with fixed start and end points, points of interest and exit
    full_path = selected_waypoints + [random.choice(final_path_waypoints)]
    print(full_path)
    
    # Intialize the whole path way
    final_path = initial_path_waypoints
        
    # Make a loop to make the path way smoother for the agente
    i = 0
    for point in full_path:
        # Select each value of the actual objective
        row = point[0]
        column = point[1]
        
        # We calculate the exit of the person in the hallway
        if final_path[-1][0] > midle_value:
            final_path = final_path +  [np.array([1625, final_path[-1][1]])]
        else:
            final_path = final_path + [np.array([275, final_path[-1][1]])]
        
        # We calcule the entry to the hallway
        # The row value is the last value of the list final_path
        final_path = final_path + [np.array([final_path[-1][0], column])]
        
        # We add the actual objective
        final_path = final_path + [point]
        
        # We iterate to next objective        
        i = i + 1
    
    return final_path

# To check for the best position in the checkout
def best_slot(checkout_map):
    # Count occupied slots per line
    occupancy = {
        line: sum(1 for x, y, pid in slots if pid != -1)
        for line, slots in checkout_map.items()
    }
    # Pick the least-crowded line
    best_line = min(occupancy, key=occupancy.get)
    # Gather free slot indices
    free_idxs = [
        idx for idx, (_x, _y, pid) in enumerate(checkout_map[best_line])
        if pid == -1
    ]
    if not free_idxs:
        print("All queues are full.")
        return
    # Pick the last free slot
    idx = max(free_idxs)
    x, y, _ = checkout_map[best_line][idx]
    
    return best_line, [x, y]

# To register a person in the position
def register_person(checkout_map, person_id, coords):
    """
    Finds the slot matching coords [x, y] in checkout_map and sets its pid to person_id.
    Returns (line_key, slot_index) if successful, or (None, None) if coords not found.
    """
    x_target, y_target = coords
    for line, slots in checkout_map.items():
        for idx, (x, y, pid) in enumerate(slots):
            if x == x_target and y == y_target:
                checkout_map[line][idx][2] = person_id
                return line, idx
    return None, None

# When a person gets done
def unregister_person(checkout_map, person_id):
    """
    Finds the slot where pid == person_id in checkout_map,
    sets that slot’s pid back to -1, and returns (line_key, slot_index).
    If the person_id is not found, returns (None, None).
    """
    for line, slots in checkout_map.items():
        for idx, (x, y, pid) in enumerate(slots):
            if pid == person_id:
                checkout_map[line][idx][2] = -1
                return line, idx
    return None, None

FINAL_Y = 850  # last position y for all lines

def find_line_and_idx_by_coords(checkout_map, coords):
    x_t, y_t = coords
    for line, slots in checkout_map.items():
        for i, (x, y, _) in enumerate(slots):
            if x == x_t and y == y_t:
                return line, i
    return None, None


def try_advance_in_line(checkout_map, person_id, coords):
    """
    If the next slot towards the cashier is free, move the person forward.
    Returns the (possibly updated) coords they should target next.
    """
    line, idx = find_line_and_idx_by_coords(checkout_map, coords)
    if line is None:
        return coords  # not found; do nothing

    slots = checkout_map[line]
    last_idx = len(slots) - 1
    if idx < last_idx and slots[idx + 1][2] == -1:
        # Move forward one slot
        unregister_person(checkout_map, person_id)
        next_coords = slots[idx + 1][:2]
        register_person(checkout_map, person_id, next_coords)
        return next_coords
    return coords  # cannot advance yet

# Pegamos toda la lista de puntos para dibujar en el mapa las trayectorias 
path_waypoints = initial_path_waypoints + middle_path_waypoints + final_path_waypoints

class Person:
    _id_counter = 1  # class‐level counter
    # Variables iniciales de la clase Person
    def __init__(self, x, y, path, infected=False, asymptomatic = False):
        # assign unique ID and increment counter
        self.id = Person._id_counter
        Person._id_counter += 1
        # Position variables
        self.position = np.array([x, y], dtype=float) # Actual position of the person
        self.velocity = np.random.rand(2) * MAX_SPEED # actual velocity of the person
        self.radius = PERSON_RADIUS # 
        self.path_index = 0 # Index of the path that is actually following
        self.path_threshold = 20  # Distance to consider waypoint reached
        self.active = True # True before checkout process
        self.path = path  # Each person gets its own path
        self.base_color = RED if infected else BLUE
        self.pulse_phase = 0  # Para el efecto de pulso
        # Checkout logic
        self.checkout = False # True in checkout process
        self.checkout_coords = [0, 0] # Position in the checkout logic
        # Logic infection
        self.infected = infected
        self.asymptomatic = asymptomatic
        if self.infected:
            self.base_color = RED
        elif self.asymptomatic:
            self.base_color = YELLOW
        else:
            self.base_color = BLUE
        # Add waiting-related attributes
        self.waiting = False
        self.wait_time = random.uniform(0.5, 2.0)  # Wait between 0.5-2 seconds        
        self.checkout_wait_time = random.uniform(*CHECKOUT_WAIT_RANGE)
        self.wait_start_time = 0
            
    def follow_path(self):
        # Normal path process
        if self.path_index < len(self.path):
            target = self.path[self.path_index]
            direction = target - self.position
            distance = np.linalg.norm(direction)
            
            # Definir los puntos donde NO debe esperar
            x_value = self.path[self.path_index][0]
            is_special_point = x_value in [275, 1625]

            if distance < self.path_threshold:
                # Solo espera en waypoints intermedios, que NO son especiales
                if (
                    2 < self.path_index < len(self.path) - 1 and  # waypoints intermedios
                    not self.waiting and
                    not is_special_point
                ):
                    # Iniciar espera
                    self.waiting = True
                    self.wait_start_time = pygame.time.get_ticks()
                    self.velocity = np.zeros(2)  # Detenerse
                elif self.waiting:
                    # Checar si terminó de esperar
                    if pygame.time.get_ticks() - self.wait_start_time > self.wait_time * 1000:
                        self.waiting = False
                        self.path_index += 1
                else:
                    # Para los puntos especiales, los dos primeros y el último: avanzar sin esperar
                    self.path_index += 1

                # Si llegó al final, desactiva el camino normal y activa checkout process
                if self.path_index >= len(self.path):
                    self.active = False
                    self.checkout = True
                    # Calculate the best new position for the person
                    line, self.checkout_coords = best_slot(checkout_map)
                    # Register in the chekout map the position of the person
                    line, idx = register_person(checkout_map, person_id=self.id, coords=self.checkout_coords )
            elif distance > 0 and not self.waiting:
                direction_normalized = direction / distance
                self.velocity += direction_normalized * 0.1
        # Checkout process
        else:
            target = np.array(self.checkout_coords)
            direction = target - self.position
            distance = np.linalg.norm(direction)

            if distance < self.path_threshold:
                line, idx = find_line_and_idx_by_coords(checkout_map, self.checkout_coords)
                if line is None:
                    # safety: re-acquire a spot
                    line, self.checkout_coords = best_slot(checkout_map)
                    register_person(checkout_map, self.id, self.checkout_coords)
                    return

                slots = checkout_map[line]
                last_idx = len(slots) - 1
                at_final_slot = (idx == last_idx) or (self.checkout_coords[1] == FINAL_Y)

                if at_final_slot:
                    # Wait only at the final slot for checkout duration
                    if not self.waiting:
                        self.waiting = True
                        self.wait_start_time = pygame.time.get_ticks()
                        self.velocity = np.zeros(2)
                    else:
                        if pygame.time.get_ticks() - self.wait_start_time > self.checkout_wait_time * 1000:
                            # Done paying; leave queue
                            self.waiting = False
                            self.checkout = False
                            unregister_person(checkout_map, self.id)
                else:
                    # INTERMEDIATE SLOTS: stand still here, do not roam
                    self.velocity = np.zeros(2)
                    self.waiting = True  # freeze at current slot

                    # If the next slot toward cashier is free, advance
                    if slots[idx + 1][2] == -1:
                        self.waiting = False  # allow movement toward the next slot
                        unregister_person(checkout_map, self.id)
                        next_coords = slots[idx + 1][:2]
                        register_person(checkout_map, self.id, next_coords)
                        self.checkout_coords = next_coords
                        # (movement vector will be set below on the next tick)
            elif distance > 0 and not self.waiting:
                # move toward current checkout target
                direction_normalized = direction / distance
                self.velocity += direction_normalized * 0.1

    def update(self, people, walls):
        if not self.active and not self.checkout:
            return

        # Always update intent/targets first
        self.follow_path()

        # If in checkout:
        if self.checkout:
            # If waiting at a slot, don't move
            if self.waiting:
                return
            # Only basic movement (no separation/wall forces)
            speed = np.linalg.norm(self.velocity)
            if speed > MAX_SPEED:
                self.velocity = (self.velocity / speed) * MAX_SPEED
            self.position += self.velocity
            self.position = np.clip(self.position, [0, 0], [WIDTH, HEIGHT])
            return

        # Normal world movement (not in checkout)
        if self.waiting:
            return
            
        # Rest of the update method remains the same...
        separation = np.zeros(2)
        for other in people:
            if other != self and other.active:
                dist = np.linalg.norm(self.position - other.position)
                if dist < SEPARATION_RADIUS:
                    separation += (self.position - other.position) / (dist + 0.0001)
        
        for wall in walls:
            dist_to_wall = self.distance_to_wall(wall)
            if dist_to_wall < WALL_MARGIN:
                separation += (self.position - wall[:2]) / (dist_to_wall + 0.0001)
        
        self.velocity += separation * 0.1
        speed = np.linalg.norm(self.velocity)
        if speed > MAX_SPEED:
            self.velocity = (self.velocity / speed) * MAX_SPEED
        self.position += self.velocity
        self.position = np.clip(self.position, [0, 0], [WIDTH, HEIGHT])
    
    def distance_to_wall(self, wall):
        x1, y1, x2, y2 = wall
        px, py = self.position
        line_length = np.linalg.norm(np.array([x2 - x1, y2 - y1]))
        if line_length == 0:
            return np.linalg.norm(self.position - np.array([x1, y1]))
        t = max(0, min(1, np.dot([px - x1, py - y1], [x2 - x1, y2 - y1]) / line_length**2))
        closest_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
        return np.linalg.norm(self.position - closest_point)
    
    def draw(self, screen):
        if not self.active and not self.checkout:
            return

        if self.infected or self.asymptomatic:
            # Tiempo animado en segundos
            time = pygame.time.get_ticks() / 250.0  # más suave que *0.005

            # Pulso de tamaño usando seno (entre 2.5x y 3.5x del radio)
            pulse_scale = 3 + 3 * math.sin(time)
            outer_radius = max(1, int(self.radius * pulse_scale))

            # Color pulsante (entre rosa claro y rojo intenso)
            pulse_color = (
                255,
                int(100 + 50 * math.sin(time)),  # valor entre 50 y 150
                int(100 + 50 * math.sin(time))
            )

            # Dibuja círculo externo
            pygame.draw.circle(screen, pulse_color, self.position, outer_radius, width=2)

        # Dibuja la persona (círculo base)
        pygame.draw.circle(screen, self.base_color, self.position, self.radius)

# Define passable walls (blue - can walk through)
passable_walls = [
    #[800, 400, 1000, 400],   # Passage between horizontal walls
    #[900, 400, 900, 600]     # Passage between vertical walls
]

# Initialize empty people list
people = []
active_count = 0
# Spawn timer
last_spawn_time = 0
current_group = 0

midle_value = 975 # half of the mall

# Main loop
running = True
while running:
    # Obtenemos el tiempo actual de la simulación
    current_time = pygame.time.get_ticks()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
    # Aparecen nuevos grupos de manera periodica
    if current_time - last_spawn_time > SPAWN_INTERVAL and active_count < NUM_PEOPLE:
        # Generar camino
        group_path = generate_random_path()
        # Generar grupo
        spawn_count = min(GROUP_SIZE, NUM_PEOPLE - active_count)
        for _ in range(spawn_count):
            # Agregamos un efecto random de contagio al grupo
            infected = random.random() < INFECTION_RATE
            asymptomatic = False # Por default asymmptomatic esta en falso
            # Entre un grupo de personas infectadas está la posibilidad de ser asimtomatico
            if infected:
                asymptomatic = random.random() < ASYMPTOMATIC_RATE 
                
            if len(people) < NUM_PEOPLE:
                # Crear nueva persona por grupo
                x = random.randint(SPAWN_AREA[0], SPAWN_AREA[1])
                y = random.randint(SPAWN_AREA[2], SPAWN_AREA[3])
                new_person = Person(x, y, group_path, infected, asymptomatic)
                people.append(new_person)
                
        active_count += spawn_count
        last_spawn_time = current_time
        current_group += 1
    
    screen.fill(WHITE)
    
    # Draw path waypoints
    for i, point in enumerate(path_waypoints):
        pygame.draw.circle(screen, GREEN, point.astype(int), 10)
        if i > 0:
            pygame.draw.line(screen, (200, 200, 200), 
                           path_waypoints[i-1], point, 2)
        
    for fila in checkout_map:
        for posicion in checkout_map[fila]:
            point = np.array([posicion[0], posicion[1]])
            pygame.draw.circle(screen, YELLOW, point.astype(int), 10)
            
            
    
    # Draw walls (solid)
    for wall in walls:
        pygame.draw.line(screen, BLACK, (wall[0], wall[1]), (wall[2], wall[3]), 5)
    
    # Draw passable walls (can walk through)
    for wall in passable_walls:
        pygame.draw.line(screen, RED, (wall[0], wall[1]), (wall[2], wall[3]), 3)
    
    # Update and draw people
    active_count = 0
    for person in people:
        person.update(people, walls)
        person.draw(screen)
        if person.active:
            active_count += 1
            
    
    # Display spawn info
    info_text = f"Active: {active_count}/{NUM_PEOPLE}  Next spawn in: {max(0, (SPAWN_INTERVAL - (current_time - last_spawn_time))//1000) }s"
    text_surface = font.render(info_text, True, BLACK)
    screen.blit(text_surface, (10, 10))
    
    pygame.display.flip()
    clock.tick(60)
    
pygame.quit()
sys.exit()
